In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from joblib import dump
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    accuracy_score,
    auc,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("../../../Datasets/SocialNetworkAds.csv")
df.head()

,Gender,Age,EstimatedSalary,Purchased
0,1,19,19000,0
1,1,35,20000,0
2,2,26,43000,0
3,2,27,57000,0
4,1,19,76000,0


In [3]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

In [ ]:
stacking = StackingClassifier()
stacking.fit(X_train, y_train)
y_pred = stacking.predict(X_test)

In [ ]:
stacking_con_matrix = confusion_matrix(y_test, y_pred)
stacking_acc = accuracy_score(y_test, y_pred)
print("Confusion Matrix", "\n", stacking_con_matrix, "\n")
print("Accuracy of Stacking Classification :", stacking_acc * 100, "\n")
print("Classification Report", "\n", classification_report(y_test, y_pred))

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6), dpi=150)
plt.style.use("dark_background")
ConfusionMatrixDisplay(stacking_con_matrix).plot(ax=ax)
plt.title("Confusion Matrix")
plt.savefig("Images/Confusion_Matrix_Stacking_Classification.png", bbox_inches="tight")

In [ ]:
y_score = stacking.predict_proba(X_test)[:, 1]
false_positive_rate, true_positive_rate, threshold = roc_curve(y_test, y_score)
roc_auc = auc(false_positive_rate, true_positive_rate)

In [ ]:
plt.figure(figsize=(6, 6), dpi=150)
plt.style.use("dark_background")
plt.plot(false_positive_rate, true_positive_rate, "r", label="AUC = %f" % roc_auc)
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.legend(loc="lower right")
plt.plot([0, 1], [0, 1], "b--")
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.savefig("Images/ROC_Curve_Stacking_Classification.png", bbox_inches="tight")

In [ ]:
print(
    "Area Under the Receiver Operating Characteristic Curve :",
    roc_auc_score(y_test, y_score),
)

In [ ]:
dump(stacking, "Model/baggingClassifier.joblib")